# Exercise 03 | Packages

Written By: Aiden Zelakiewicz (asz39@cornell.edu)

Inspired By: [2022 Cornell REU Python Workshop Part 03](https://github.com/CUAstro-REU-Python-Workshop/2022-workshop)

In this exercise, we will go over many useful Python packages for astronomy and some of their features. Three of the big package we will overview are `numpy`, `astropy`, and `scipy`. For `numpy` in particular, I will overview some useful features of the package we did not go over in the prior exercises. We will also splash in some `matplotlib` to visualize many of the concepts we will go over. All of these packages work well together, so you should notice some overlap as we proceed.

### Goals for this exercise

1. Manipulating Numpy arrays

2. Astropy Units

3. Astropy I/O and Fits files

4. Astropy Coordinates

5. Scipy Optimization, Interpolation, Integration

## Numpy

In [ ]:
import numpy as np

### Creating Arrays

There are many ways to create various arrays in `numpy` that you certainly will find yourself using frequently. Below I am just going to show a few examples of the various ways to create organized and random arrays of numbers.

With randomness within `numpy`, you can set a seed so that others can get the same set of random numbers. You do this by setting `np.random.seed()` to whatever you would like the seed to be.

In [ ]:
# Equally spaced points
x_lin = np.linspace(0, 10, 100)

# Logarithmically spaced points
x_log = np.logspace(0, 1, 100)

# Set the seed for reproducibility
np.random.seed(42)

# Random points in range [0, 1)
x_rand = np.random.rand(100)

# Random integers
x_randint = np.random.randint(0, 10, 100)

# Normal distribution about a value (0) with a standard deviation of 1
x_normal = np.random.normal(loc=0, scale=1, size=100) # Explicitly setting loc and scale for clarity

# Zero array
x_zeros = np.zeros(100)

# Ones array
x_ones = np.ones(100)

### Manipulating Arrays

Often you have an array that has varying types of data, some of which 

Let's practice some of these techniques on real data from observations at the United Kingdom Infrared Telescope (UKIRT). In this folder is a single CMD from yours truly which contains the $K$-band and $H-K$ color, stored in `ukirt.csv`.

In [ ]:
# Read in UKIRT data, has header K, H-K
ukirt = np.loadtxt('ukirt.csv', delimiter=',', skiprows=1)

import matplotlib.pyplot as plt
plt.style.use('./az-paper-twocol.mplstyle')

# We will use this more than once, so let's define a function!
# Remember DRY: Don't Repeat Yourself
def plot_cmd(array, show=True):
    fig, ax = plt.subplots(1, 1, figsize=(6,6))
    ax.scatter(array[:,1], array[:,0], s=.5, c='#260101') # You can also use hex colors
    ax.set_xlabel(r'$H-K$')
    ax.set_ylabel(r'$K$')
    ax.set_xlim(0., 3.)
    ax.set_ylim(18., 11.) # This reverses the y-axis

    if show:
        plt.show()

plot_cmd(ukirt)

#### Where
This field is used to calculate the extinction (amount of light blocked from observer) in the Milky Way. Let's say we want to isolate stars with $K$-band magnitude between [12,16]. In vanilla Python, one might think to loop (or list comprehension) the entire dataset. Luckily, `numpy` has useful functions like `np.where()` which can do exactly this thing! It will return the indices where a specific condition is met on an array.

In [ ]:
# Let's isolate the stars with K magnitudes between 12 and 16
# We can combine conditions with & (and) and | (or)
inds = np.where((ukirt[:,0] <= 16) & (ukirt[:,0] >= 12))

# Isolate the data
ukirt_sub = ukirt[inds]

plot_cmd(ukirt_sub) # Yay function!

A good logical way to understand what happens is your condition creates an array of `booleans`, which you then pass to `np.where()` to return the index at which those `booleans` are `True`. 

#### Argsort

You may need to sort your data, which `np.argsort()` may be useful. It return the indices for which the data would be sorted.

In [ ]:
# Argsort returns the indices that would sort the array

# Sort by K magnitude
inds = np.argsort(ukirt[:,0])
ukirt_sorted = ukirt[inds]

print(ukirt_sorted)

#### Percentile

Getting the value at a specific percentile in an `ndarray` is very useful for a number of cases. One I imediately think of is setting colorbar scales for data, where outliers can make your figure unreadable! Let's find the 90\% value of $H-K$ color using `np.percentile()`.

In [ ]:
# Get stars above 10% in H-K using np.percentile
np.percentile(ukirt[:,1], 90)

#### <---TO DO--->

Quick wake up activity!
Combine `np.where()` and `np.percentile()` to get all the stars above the 10th percentile in $H-K$ color AS WELL AS all of the stars within the [12,16] range of $K$-band magnitude.
Then plot these stars using the function we made!
This is to show the power of combining various `numpy` functions to quickly achieve a somewhat difficult result!

If this seems like a lot to do, remember you can break it up into tiny chunks!

In [ ]:
### YOUR CODE HERE

## Astropy

### Units and Constants

One of the most useful features of `astropy` is its ability to handle units (the module is named `units`). If you assign a variable a unit, you can ask `astropy` to convert it do a different standard or do calculations without having to convert things yourself. Don't tell my professors, but I haven't calculated unit conversions by hand in ages thanks to `astropy`. This module alone made using Python my go-to calculator, especially since they work with `numpy` functions!

Another module, which goes hand-in-hand with `astropy.units` is `astropy.constants`. Astropy has a lot of standard constants set, such as Boltzman's constant or Plank's constant.

There is a slight learning curve with using `astropy.units`, as you need to know how they define all of the units. Most, however, are very intuitive and you can always check the docs if you're confused!

In [ ]:
# Usually people load astropy modules in seperately, rather than the entire package
import astropy.units as u
import astropy.constants as c # a common alias for this is 'const', I'm just lazy

# Create a unit of length
length = 26.2 * u.m

print(length)

print(length.to(u.cm))

Notice above how we used the `.to()` attribute to change the units. This is incredibly useful when you need to do math and work out units!

In [ ]:
# Get the recieved flux from a star
lum = 0.85 * u.Lsun
dist = 10 * u.pc

# F = L / (4 * pi * d^2)
flux = lum / (4 * np.pi * dist**2)
print(f"Flux: {flux.to(u.W / u.m**2):0.2e}")

You might be wondering what I did to the string above. That is called "formatting" (denoted by `:`), which just alters the way the value is displayed. In this case, I asked to go to 2 decimal places (the `0.2`) and be put in scientific notation (the `e`).

Let's also use some constants to get the force of gravity between the Earth and Sun!

In [ ]:
# Using constant c.G
Fg = c.G * c.M_earth * c.M_sun / c.au**2

print(f"{Fg.to(u.N):0.2e}")

### Reading Files

While the `astropy.io` module contains many useful functions like `readascii()`, the most important tool is the ability to read `fits` files. For the uninformed, `fits` files are a common file format used within astronomy. You can only avoid `fits` for so long before being forced to use them (I tried). We will download a sample fits file from `astropy` and open it using `fits.open()`.

In [ ]:
from astropy.io import fits

# Let's read in a sample data file from astropy
from astropy.utils.data import download_file
image_file = download_file('http://data.astropy.org/tutorials/FITS-images/HorseHead.fits', cache=True)

# Open the file with fits.open()
hdu_list = fits.open(image_file)

# Structure of the file
hdu_list.info()

`fits` files have headers, and data is usually stored in the `Primary` block. You can access different blocks through indexing.

In [ ]:
# Get the data
data = hdu_list[0].data

# You can also do it this way if you don't want header information
data = fits.getdata(image_file)

# Access the header, which has metadata
header = hdu_list[0].header

We can access the header keys using `header.keys` and call them similarly to a dictionary.

In [ ]:
print(header.keys)

In [ ]:
# Exposure time in minutes
header['EXPOSURE']

Yikes thats a lot of information! We can now close the file to free up memory since we have the data saved.

In [ ]:
# Close the file
hdu_list.close()

Like we did in the `matplotlib` exercise, let's plot this image using `plt.imshow()`.

In [ ]:
# Plot the image with imshow
plt.imshow(data, cmap='gray')
plt.colorbar()
plt.show()

### Coordinates

Astronomy has various coordinate systems with nontrivial transformations between them. `Astropy` can handle these coordinate systems for us and provide a lot of other cool functionalities like proper axis plotting! Most often you will work through `astropy.coordinates.SkyCoord()` to do coordinate related activites.

Let's extract the Right Ascension (RA) and Declination (Dec) of the horsehead image above and create a `SkyCoord` for it.

In [ ]:
from astropy.coordinates import SkyCoord

# Get RA and Dec of Horsehead Nebula
RA = header['PlateRA']; Dec = header['PlateDec'] # These are in degrees

# Create a SkyCoord object
coords = SkyCoord(ra=RA*u.deg, dec=Dec*u.deg, frame='icrs') # ICRS is the default frame for RA and Dec

In typical `astropy` fashion, we can convert these to other units or other reference frames.

In [ ]:
# Print the coordinates
print(coords.to_string('hmsdms'))

# Can also just get RA or Dec
print(coords.ra.hms)
print(coords.ra.deg)

# Get the galactic coordinates
gal_coords = coords.galactic
print(gal_coords)

It is useful to know that `SkyCoords` can take an array of location data, which is substantially faster than creating individual SkyCoordinate objects. You can then access the individual coordinates using indexing.

In [ ]:
# Array of coordinates
ra = np.array([10, 20, 30]) * u.deg
dec = np.array([45, 50, 55]) * u.deg

# Create a SkyCoord object
coord_arr = SkyCoord(ra=ra, dec=dec)

print(coord_arr)
print(coord_arr[1])

#### World Coordinate System

Another way of getting coordinates from `fits` files is using the World Coordinate System, or `astropy.WCS`. By giving `astropy` the header of a `fits` file, we can get its location information and plot projections! This is done by setting the `projection` argument of `plt.subplot()` to the `wcs` from `astropy`.

In [ ]:
# Create WCS from header
from astropy.wcs import WCS

horsehead_wcs = WCS(header)

# Plot the image with imshow and WCS
plt.subplot(projection=horsehead_wcs)

plt.imshow(data, cmap='gray')
plt.colorbar()

# Add gridlines
plt.grid(color='white', ls='dotted')

plt.xlabel('Right Ascension')
plt.ylabel('Declination')

plt.show()

## Scipy

`Scipy` is an incredibly powerful package that is useful for various mathematical and statistical problems. Like `astropy`, people usually import just the modules they need rather than the entire `scipy` library.

### Curve Fitting/Optimization

One of the key features of `scipy` is the ability to fit functions to data. One of the most basic versions of this is `scipy.optimize.curve_fit()`, which given a function and data will return the optimized set of parameters in the function along with the covariance matrix.

In [ ]:
# Example taken from scipy documentation
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
from scipy.optimize import curve_fit

# define our model
def func(x, a, b, c):
  return a * np.exp(-b * x) + c

# generate some data
xdata = np.linspace(0, 4, 50)
y = func(xdata, 2.5, 1.3, 0.5)

# add noise to the data
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise

# Run curve_fit
# popt is the optimized parameters
# pcov is the covariance matrix
popt, pcov = curve_fit(func, xdata, ydata)

print(popt)

We can visualize our curve fit using `matplotlib`.

In [ ]:
plt.figure(figsize=(6,4))

# Plot the data and the fit
plt.plot(xdata, ydata, label='data', marker='o', ls='none')

# The *popt unpacks the tuple into the arguments
plt.plot(xdata, func(xdata, *popt), label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

That was super easy! Often, you will be aiming to minimize functions when working with data. This is common with minimizing residuals or a loss function, which can all be done through `scipy.optimize.minimize()`.

The UKIRT data we were working and chopped earlier has a population of stars in them called Red Clump (RC) stars. These are the dense region of stars in the CMD we plotted earlier. We can try and recover the magnitude of the RC stars in this field by optimizing a gaussian.

In [ ]:
# Let's minimize a function
from scipy.optimize import minimize

inds = np.where((ukirt[:,0] <= 16) & (ukirt[:,0] >= 12) & (ukirt[:,1] >= np.percentile(ukirt[:,1], 20)))

# Isolate the data
ukirt_sub = ukirt[inds]

# Bin the data in K-magnitude
bins = np.linspace(12, 16, 20)
hist, bin_edge = np.histogram(ukirt_sub[:,0], bins=bins)

# Define a Gaussian function with exponential
def gaussian(x, a, b, c, d, e):
    return a * np.exp(-0.5 * ((x - b) / c)**2) + d*np.exp(e*(x-b))

# Define the chi-squared function
def chi2(params, x, y, yerr):
    return np.sum((y - gaussian(x, *params))**2 / yerr**2)

# Initial guess for the parameters
params0 = [100, 13, 1, 10, -0.5]

# Run the minimization. Various methods can be used here, but we'll use the default
result = minimize(chi2, params0, args=(bin_edge[:-1], hist, np.sqrt(hist)))

print(result.x)

# plot hist
plt.figure(figsize=(6,4))
plt.step(bin_edge[:-1], hist, where='mid', label='Binned Data')
plt.plot(bin_edge[:-1], gaussian(bin_edge[:-1], *result.x), color='r', label='Fit')

plt.xlabel('K Magnitude')
plt.ylabel('Number of Stars')
plt.legend()
plt.show()

### Interpolation

Suppose you have a sparsely sampled data set, and you want to estimate how the data behaves with higher resolution or estimate intermediate values of the data. This is where `scipy` comes to help with its various interpolation functions, such as `interp1d()`.

In [ ]:
from scipy.interpolate import interp1d

# Define the function
x = np.linspace(0, 10, 10)
y = np.sin(x)

# Linear interpolation
f = interp1d(x, y) # Creates a function that can be called with new x values
x_new = np.linspace(0, 10, 100)
y_new = f(x_new)

# Plot the data
plt.figure(figsize=(6,4))

# zorder sets the order of the plot, so the data is on top
plt.plot(x, y, 'o', label='data', zorder=10)
plt.plot(x_new, y_new, label='linear interp')

plt.xlabel('x')
plt.ylabel('y (sin(x))')

plt.legend()
plt.show()

#### <---TO DO--->

Well that didn't look great. How about we try a different interpolation method that might do a better job. In your browser, search up `scipy`'s 'cubic' interpolater and see how it works (it should be very similar!). Import the function and implement it on the same data from above. Plot the results along with the linear interpolation to see the improvement! 

In [ ]:
### YOUR CODE HERE

### ODE Solving

`scipy` can also be used for solving ODEs. Lets just do a quick practice on $\frac{\delta y}{\delta t} = -y$ using `scipy.integrate.solve_ivp()`. This solves the initial value problem given a range and initial condition.

In [ ]:
from scipy.integrate import  solve_ivp

# Define the function to integrate
def dydt(t, y):
    return -y

# Define the initial conditions
y0 = 1

# Define the time range
t_span = [0, 10]

# Solve the ODE
sol = solve_ivp(dydt, t_span, [y0])

# Plot the solution
plt.figure(figsize=(6,4))
plt.plot(sol.t, sol.y[0], 'ok')
plt.xlabel('t')
plt.ylabel('y')
plt.show()